Fuzzy Language in Data Science

Request: There is are $1000 left in the budget. Find the best customers who haven't purchased online in the last 90 days and send them a coupon for online purchases

In [1]:
import pandas as pd
import datetime as dt

In [2]:
# our dataset consists of 125K transactions with a customer id, transaction date and transaction amount
data = pd.read_csv('rfm_xmas19.txt', parse_dates=['trans_date'])

In [3]:
data.shape

(125000, 3)

In [4]:
data.head(1)

,customer_id,trans_date,tran_amount
0,FM5295,2017-11-11,35


In [5]:
# gropuing the data by customer_id
group_by_customer = data.groupby('customer_id')

# getting the last transaction for each customer
last_transaction = group_by_customer['trans_date'].max()

In [6]:
# group_by_customer

In [7]:
type(last_transaction)

pandas.core.series.Series

In [8]:
# converting our dataset into a dataframe
best_churn = pd.DataFrame(last_transaction)

In [9]:
best_churn.head()

,trans_date
customer_id,
FM1112,2019-10-14
FM1113,2019-11-09
FM1114,2019-11-12
FM1115,2019-12-05
FM1116,2019-05-25


In [10]:
best_churn.dtypes

trans_date    datetime64[ns]
dtype: object

In [11]:
# best_churn['trans_date'] = dt.date(best_churn['trans_date'])

In [12]:
# date time object to set the cut-off date
cutoff_day = dt.datetime(2019, 10, 16)

In [13]:
# function to separate the cutoff date
def churn(tran):
   # date = dt.date(tran)
    if tran < cutoff_day:
        return 1
    else:
        return 0

In [14]:
# best_churn['churned'] = best_churn['trans_date'].apply(churn)

In [15]:
# doing the same as the function but with a lambda function
best_churn['churned'] = best_churn['trans_date'].apply(lambda x: 1 if x < cutoff_day else 0)

In [16]:
best_churn.head(3)

,trans_date,churned
customer_id,,
FM1112,2019-10-14,1
FM1113,2019-11-09,0
FM1114,2019-11-12,0


In [17]:
best_churn.dtypes

trans_date    datetime64[ns]
churned                int64
dtype: object

In [18]:
# defining best customers as those with the most purchases with highest transactions
# number of transactions 
best_churn['num_transactions'] = group_by_customer.size()
best_churn.head(3)

,trans_date,churned,num_transactions
customer_id,,,
FM1112,2019-10-14,1,15
FM1113,2019-11-09,0,20
FM1114,2019-11-12,0,19


In [19]:
# getting the total amount spent by customer
best_churn['amount_spent'] = group_by_customer['tran_amount'].sum()
best_churn.head(2)

,trans_date,churned,num_transactions,amount_spent
customer_id,,,,
FM1112,2019-10-14,1,15,1012
FM1113,2019-11-09,0,20,1490


In [20]:
# dropping transaction date
best_churn = best_churn.drop(columns=['trans_date'])

In [21]:
best_churn.head(2)

,churned,num_transactions,amount_spent
customer_id,,,
FM1112,1,15,1012
FM1113,0,20,1490


In [22]:
# best_churn[['num_transactions','amount_spent']].describe()

In [23]:
# let's review the ranges of num_transactions and amount_spent
best_churn[['num_transactions','amount_spent']].describe().loc[['min', 'max']]

,num_transactions,amount_spent
min,4.0,149.0
max,39.0,2933.0


In [24]:
# creatinga  scaled column for num_transactions to give it the same weight as amount spent (x-min(x)/max(x)-min(x))
best_churn['scaled_tran'] = (best_churn['num_transactions'] - best_churn['num_transactions'].min()) / (best_churn['num_transactions'].max() - best_churn['num_transactions'].min())    

In [25]:
# creatinga  scaled column for num_transactions to give it the same weight as amount spent (x-min(x)/max(x)-min(x))
best_churn['scaled_amount'] = (best_churn['amount_spent'] - best_churn['amount_spent'].min()) / (best_churn['amount_spent'].max() - best_churn['amount_spent'].min())    

In [26]:
best_churn.head(2)

,churned,num_transactions,amount_spent,scaled_tran,scaled_amount
customer_id,,,,,
FM1112,1,15,1012,0.314286,0.309986
FM1113,0,20,1490,0.457143,0.481681


In [27]:
# creating a score column that is (1/2  num_transactions) + (1/2 x amount_spent)
best_churn['score'] = ((.5 * best_churn['scaled_tran']) + (.5 * best_churn['scaled_amount'])) * 100

In [28]:
best_churn.head(2)

,churned,num_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
FM1112,1,15,1012,0.314286,0.309986,31.213567
FM1113,0,20,1490,0.457143,0.481681,46.941195


In [29]:
# sorting our dataframe by the highest score to lowest
best_churn.sort_values(by='score', ascending=False, inplace=True)

In [30]:
best_churn.head()

,churned,num_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
FM4424,0,39,2933,1.000000,1.000000,100.000000
FM4320,1,38,2647,0.971429,0.897270,93.434934
FM3799,1,36,2513,0.914286,0.849138,88.171182
FM5109,0,35,2506,0.885714,0.846624,86.616892
FM3805,1,35,2453,0.885714,0.827586,85.665025


In [31]:
# we need to decide a treshhold to select our best churned customers
# a little less than half are churned customers
best_churn['churned'].value_counts()

0    3671
1    3218
Name: churned, dtype: int64

In [32]:
# figuring out a strategy to provide $1,000 worth of coupons

mean_tran = data['tran_amount'].mean()
mean_tran

64.991912

In [33]:
# 30% of the mean transactions
coupon = mean_tran * 0.3
coupon

19.4975736

In [34]:
# getting the number of customers that we will market the coupons too
nr_of_customers = 1000/coupon
nr_of_customers

51.28843314123969

Based on this analyiss we will offer 50 customers an on-line coupon for $50 (rounding the amounts above)

In [35]:
# creating a mask for churned customers and selecting the top 50
mask = best_churn['churned'] == 1
top_50_churned = best_churn[mask].head(50)

In [36]:
# exporitng the results to a csv file
top_50_churned.to_csv('best_customers.txt')